In [52]:
from feat import Detector # py-feat

In [53]:
face_model = 'img2pose'
landmark_model='mobilenet' 
facepose_model='img2pose-c'


detector = Detector(face_model=face_model, 
                    landmark_model=landmark_model, 
                    au_model='xgb', 
                    emotion_model='resmasknet', 
                    facepose_model=facepose_model,
                    identity_model='facenet', 
                    device='cuda', 
                    n_jobs=1, 
                    verbose=False,
                    )

INFO:root:Loading Face model: img2pose
INFO:root:Loading Facial Landmark model: mobilenet
INFO:root:Loading facepose model: img2pose-c
INFO:root:Loading AU model: xgb
INFO:root:Loading emotion model: resmasknet
INFO:root:Loading Identity model: facenet


In [54]:
video_path = "F:\\RAVDESS\\Actor_14\\01-01-02-02-02-02-14.mp4"
print(video_path)
output_filename = f'F:\\tracked_au\\{video_path[-24:-4]}.mp4'
print(output_filename)

F:\RAVDESS\Actor_14\01-01-02-02-02-02-14.mp4
F:\tracked_au\01-01-02-02-02-02-14.mp4


In [55]:
print("running py-feat detector")
# run detector
video_prediction = detector.detect_video(video_path, 
                                         skip_frames=None, 
                                         output_size=(720, 1280), 
                                         batch_size=5, 
                                         num_workers=0, 
                                         pin_memory=False, 
                                         face_detection_threshold=0.9, 
                                         face_identity_threshold=0.8
                                         )


running py-feat detector


  0%|          | 0/23 [00:00<?, ?it/s]INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:VideoDataset: RESCALING WARNING: from (720, 1280) to output_size=(720, 1280)
INFO:root:detecting faces...
INFO:root:detecting landmarks...
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 554, 554]) to 224
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 554, 554]) to 224
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 554, 554]) to 224
INFO:root:RESCALING WARNING: image_operations

In [57]:
video_prediction.to_csv(f'F:\\tracked_au\\{video_path[-24:-4]}.csv')

In [56]:
import numpy as np
from feat.plotting import plot_face
import matplotlib.pyplot as plt
import cv2
import os


muscles = {'all': 'heatmap'}
total_frame = video_prediction.aus.shape[0]
output_dir = "F:\\tracked_au\\frame"

# Create directory for frames if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize an empty list to store frames that raise ValueError
error_frames = []

# Generate and save frames
for frame in range(total_frame):
    try:
        print(frame)
        aus = np.array(video_prediction.aus.iloc[frame])
        print(aus)
        ax = plot_face(au=aus, muscles=muscles, title=f'{frame=}')  # line to handle
        plt.savefig(f'{output_dir}/frame_{frame:04d}.png')
        plt.close()
    except ValueError as e:
        print(f"ValueError encountered at frame {frame}: {e}")
        error_frames.append(frame)

# Optionally, print or save the error_frames list
print("Frames that raised ValueError:", error_frames)

# exclude error frames
full_range = range(total_frame)
valid_frames = [frame for frame in full_range if frame not in error_frames]

# Video settings
fps = 30
frame_array = []
frame_size = None

# Read frames and add to the video array
for frame in valid_frames:
    filename = f'{output_dir}/frame_{frame:04d}.png'
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    frame_array.append(img)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

# Write frames to the video
for frame in frame_array:
    out.write(frame)

# Release everything if job is finished
out.release()
cv2.destroyAllWindows()

print("Video created successfully")
print("Frames that raised ValueError:", error_frames)

0
[0.18453404 0.288726   0.30462244 0.42502552 0.2167253  0.
 0.16544642 0.0016033  1.         0.12118208 0.6103023  0.07655756
 0.5392229  0.         0.40061074 0.23090847 0.2225768  0.26921272
 0.10933189 0.02326062]


AttributeError: 'PLSRegression' object has no attribute '_predict_1d'

In [ ]:
from feat.plotting import plot_face
import numpy as np

In [ ]:
video_prediction.aus.head

<bound method NDFrame.head of            AU01      AU02      AU04      AU05      AU06  AU07      AU09  \
frame                                                                     
0      0.914365  0.665577  0.190557  0.584313  0.072784   0.0  0.086320   
1      0.906166  0.592989  0.132303  0.576765  0.077927   0.0  0.093537   
2      0.923769  0.593716  0.124901  0.454841  0.073544   0.0  0.095412   
3      0.904651  0.677646  0.134747  0.589844  0.094573   0.0  0.091658   
4      0.904743  0.644907  0.098356  0.445532  0.091564   0.0  0.095016   
...         ...       ...       ...       ...       ...   ...       ...   
114    0.916465  0.740485  0.177407  0.404966  0.085242   0.0  0.129527   
115    0.896537  0.635496  0.115616  0.403953  0.072244   0.0  0.137503   
116    0.923014  0.607706  0.070101  0.393016  0.089199   0.0  0.135164   
117    0.910651  0.563698  0.092361  0.413820  0.082782   0.0  0.131110   
118    0.909673  0.739993  0.090297  0.411361  0.075129   0.0  0.13946

In [ ]:

frame = 0
aus = np.array(video_prediction.aus.iloc[frame])
print(aus)
print(aus.shape)
ax = plot_face(au=aus, title=f'{frame=}')

[0.9143645  0.6655766  0.19055688 0.5843133  0.07278425 0.
 0.08631986 0.1596458  0.         0.0667465  0.37487695 0.47726965
 0.37931907 0.         0.42618906 0.39251116 0.01439848 0.3136797
 0.1752088  0.05640208]
(20,)


AttributeError: 'PLSRegression' object has no attribute '_predict_1d'

In [ ]:

# # Constants for video settings
# VIDEO_WIDTH, VIDEO_HEIGHT = 1280, 720  # Desired resolutionFalse
# DPI = 100  # Resolution in dots per inch

# print("generating overlay for each frame")
# # generate overlay for each frame
# figs = video_prediction.plot_detections(faces='landmarks', 
#                                           faceboxes=True, 
#                                           muscles=False, 
#                                           poses=True, 
#                                           gazes=False, 
#                                           add_titles=False, 
#                                           au_barplot=False, 
#                                           emotion_barplot=False, 
#                                           plot_original_image=True
#                                           );

# # Create a video writer object, specifying the output file, codec, and framerate
# writer = imageio.get_writer(output_filename, fps=30, codec='libx264', format='FFMPEG', macro_block_size=None)

# print("generating video")
# # Loop through each figure in the list, adjust size, save it to a buffer, and append it to the video
# for fig in figs:
#     # Set the figure size to correspond to the desired pixel dimensions
#     fig.set_size_inches(VIDEO_WIDTH / DPI, VIDEO_HEIGHT / DPI)
    
#     # Use a buffer to save the figure
#     buf = io.BytesIO()
#     fig.savefig(buf, format='png', dpi=DPI)
#     buf.seek(0)
    
#     # Read the image from the buffer and add it to the video
#     image = imageio.imread(buf)
#     writer.append_data(image)
    
#     # Close the buffer
#     buf.close()

# # Close the writer to finalize the video
# writer.close()

# print(f'Video saved as {output_filename}')
